<a href="https://colab.research.google.com/github/shimul1725/Stock-Market-Performance-Analysis/blob/main/Stock_Market_Performance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# we can collect real-time stock market data using the yfinance API:
import pandas as pd
import yfinance as yf
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-03-27  159.940002  160.770004  157.869995  158.279999  158.061371   
       2023-03-28  157.970001  158.490005  155.979996  157.649994  157.432236   
       2023-03-29  159.369995  161.050003  159.350006  160.770004  160.547943   
       2023-03-30  161.529999  162.470001  161.270004  162.360001  162.135742   
       2023-03-31  162.440002  165.000000  161.910004  164.899994  164.672226   

                     Volume  
Ticker Date                  
AAPL   2023-03-27  52390300  
       2023-03-28  45992200  
       2023-03-29  51305700  
      

In [13]:
#In this dataset, the Date column is the index column in the DataFrame. We need to reset the index before moving forward:

df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-03-27  159.940002  160.770004  157.869995  158.279999   
1   AAPL 2023-03-28  157.970001  158.490005  155.979996  157.649994   
2   AAPL 2023-03-29  159.369995  161.050003  159.350006  160.770004   
3   AAPL 2023-03-30  161.529999  162.470001  161.270004  162.360001   
4   AAPL 2023-03-31  162.440002  165.000000  161.910004  164.899994   

    Adj Close    Volume  
0  158.061371  52390300  
1  157.432236  45992200  
2  160.547943  51305700  
3  162.135742  49501700  
4  164.672226  68749800  


In [14]:
#Now let’s have a look at the performance in the stock market of all the companies:

import plotly.express as px
fig = px.line(df, x='Date',
              y='Close',
              color='Ticker',
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [15]:
#Now let’s look at the faceted area chart, which makes it easy to compare the performance of different companies and identify similarities or differences in their stock price movements:

fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [16]:
#Let us now look at moving averages, which are a great tool to find patterns and trends in each company's stock price movements over time:

df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10      MA20
0          NaN       NaN
1          NaN       NaN
2          NaN       NaN
3          NaN       NaN
4          NaN       NaN
..         ...       ...
57  182.012001  178.8445
58  182.555000  179.3370
59  183.030000  179.8250
60  183.948000  180.5970
61  184.558998  181.3390

[62 rows x 2 columns]
Moving Averages for GOOG
        MA10      MA20
186      NaN       NaN
187      NaN       NaN
188      NaN       NaN
189      NaN       NaN
190      NaN       NaN
..       ...       ...
243  124.603  124.3735
244  124.325  124.4035
245  123.660  124.1730
246  123.753  124.2020
247  123.788  124.2710

[62 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
62          NaN         NaN
63          NaN         NaN
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
..          ...         ...
119  333.896002  329.699002
120  334.107001  330.684502
121  334.095001  331.303502
122  335.728000  332.52

In [17]:
#Visualize the moving averages of all companies
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'],
                  title=f"{ticker} Moving Averages")
    fig.show()

In [18]:
#Volatility is a measure of how much and how often the stock price or market fluctuates over a given period of time, Here’s how to visualize the volatility of all companies:

df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility',
              color='Ticker',
              title='Volatility of All Companies')
fig.show()

In [19]:
#Now let’s analyze the correlation between the stock prices of Apple and Microsoft:

# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT',
                 trendline='ols',
                 title='Correlation between Apple and Microsoft')
fig.show()

In [21]:
#Now let’s analyze the correlation between the stock prices of Netflix and Google:

# create a DataFrame with the stock prices of netflix and Google
netflix = df.loc[df['Ticker'] == 'NFLX', ['Date', 'Close']].rename(columns={'Close': 'NFLX'})
google = df.loc[df['Ticker'] == 'GOOG', ['Date', 'Close']].rename(columns={'Close': 'GOOG'})
df_corr = pd.merge(netflix, google, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='NFLX', y='GOOG',
                 trendline='ols',
                 title='Correlation between netflix and google')
fig.show()